<a href="https://colab.research.google.com/github/nigitha-16/Violence-Detection-in-Surveillance-Video/blob/master/train_violence_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizers import SGD, Adagrad,Adam
import cv2
import numpy as np
import os
from numpy import savetxt
from numpy import loadtxt
from keras.regularizers import l2

import matplotlib.pyplot as plt



In [ ]:
initializer = tf.keras.initializers.GlorotNormal()


In [ ]:
def build_model():
  model = Sequential()
  model.add(Dense(512, input_dim=4096,kernel_regularizer='l2',kernel_initializer=initializer,activation='relu'))
  #model.add(Dropout(0.6))
  model.add(Dense(32,kernel_regularizer='l2',kernel_initializer=initializer))
  #model.add(Dropout(0.6))
  model.add(Dense(1,activation='sigmoid'))
  return model

Load features

In [ ]:
### load the csv file containing features ###
ar=loadtxt('/content/drive/My Drive/train_anom_nom_feat.csv',delimiter=',')

ar_anom=ar[:34*32]
ar_nom=ar[34*32:]


In [ ]:
def load_dataset_Train_batch(ar_nom=ar_nom,ar_anom=ar_anom,batch_size=8):

  n_vid_each=int(batch_size/2) #number of videos in each category

  count=-1

  #### generate random index ####
  Abnorm_iter = np.random.permutation(34)
  Abnorm_iter = Abnorm_iter[:n_vid_each] 
  Norm_iter = np.random.permutation(40)
  Norm_iter = Norm_iter[:n_vid_each]   

  Features = []

 
  for f in Abnorm_iter:

    feat = ar_anom[f*32:f*32+32] ####each video has 32 segments
    count = count + 1
    if count == 0:
        Features = feat
    if count > 0:
        Features = np.append(Features, feat,axis=0)
  

  AllFeatures = Features
  count=-1
  for f in Norm_iter:
    feat = ar_nom[f*32:f*32+32]  ####each video has 32 segments
    count = count + 1
    if count == 0:
        Features = feat
    if count > 0:
        Features = np.append(Features, feat,axis=0)
  

  ###### stack the normal features over the abnormal #####
  AllFeatures = np.vstack((AllFeatures, Features))

  ######## labels #######
  l_0 = np.ones(32*n_vid_each, dtype='uint8') ## 1 for abnormal
  l_1= np.zeros(32*n_vid_each, dtype='uint8') ## 0 for normal
  all_labels=np.append(l_0,l_1,axis=0)
  return AllFeatures,all_labels

In [ ]:
##### define the loss function ##########
def custom_loss(y_true, y_pred):
    
    'Custom Objective function'
    
    y_true = tf.reshape(y_true,[-1])
    y_pred = tf.reshape(y_pred,[-1])

    n_seg = 32  
    batch_size = 8
    n_vid_each = int(batch_size / 2)
    total_feat=32*batch_size


    s_max = tf.ones_like(y_pred) # holds the highest scoring instance
    s_l1=tf.cast(tf.ones_like(y_true),'float32')  # holds l1 score
    s_l2 =tf.cast(tf.ones_like(y_true),'float32') # holds the l2 score

    for ii in range(batch_size):
        # For Labels
        mm = y_true[ii * n_seg:ii * n_seg + n_seg]
      
        # For Features scores
        seg_score = y_pred[ii * n_seg:ii * n_seg + n_seg]
        s_max = tf.concat([s_max, tf.reshape(tf.math.reduce_max(seg_score),[1])],0) # maximum score of scores 
        s_l1 = tf.concat([s_l1, tf.reshape(tf.math.reduce_sum(seg_score),[1])],0)   # sum of scores of all instances 
        z1 = tf.ones_like(seg_score)
        z2 = tf.concat([z1, seg_score],0)
        z3 = tf.concat([seg_score, z1],0)
        z = z2[31:] - z3[:33]
        z = z[1:32]
        z = tf.math.reduce_sum(tf.math.square(z))
        s_l2 = tf.concat([s_l2, tf.reshape(z,[1])],0)

    
    sub_score = s_max[total_feat:]  # We need this step since we have used tf.ones_like
    
 
    s_l1 = s_l1[total_feat:] # We need this step since we have used tf.ones_like
    s_l1 = s_l1[:n_vid_each]
    s_l2 = s_l2[total_feat:] # We need this step since we have used tf.ones_like
    s_l2 = s_l2[:n_vid_each]




    Sub_Nor = sub_score[n_vid_each:] # Maximum Score for each of abnormal video
    Sub_Abn = sub_score[:n_vid_each] # Maximum Score for each of normal video

    
    z=tf.maximum(tf.reduce_sum(1-Sub_Abn+Sub_Nor),0)+  0.008*tf.math.reduce_sum(s_l1) + 0.008*tf.math.reduce_sum(s_l2) ### final loss 
    
    return z


Train

In [ ]:
adagrad=Adagrad(lr=0.01, epsilon=1e-07)
model=build_model()

model.compile(loss=custom_loss, optimizer=adagrad)

print("Starting training...")

loss_graph =[]
num_iters = 2000
total_iterations = 0
batch_size=8


for it_num in range(num_iters):


    inputs, targets=load_dataset_Train_batch(ar_nom,ar_anom,batch_size)  # Load normal and abnormal video C3D features
    batch_loss =model.train_on_batch(inputs, targets)
    
    loss_graph = np.hstack((loss_graph, batch_loss))
    if total_iterations%200 ==0:
      model.save_weights('/content/drive/My Drive/model_weights_violence_detection.h5')
      print(batch_loss)
    total_iterations += 1
plt.plot(loss_graph)